# Lab 2: Modelamiento multidimensional

# Nombre: Joel Molina
# NRC: 8903

## Librerias

In [1]:
import pandas as pd 
import numpy as np 
import sqlalchemy
from sqlalchemy import create_engine
import pyodbc
import sqlite3
from datetime import datetime

## Conexión a SQLite con la base de datos Chinook

In [2]:
engine = create_engine('sqlite:///chinook.db') #creación de la conexión con la bdd chinook
engine

Engine(sqlite:///chinook.db)

## Creación del DataFrame de las fechas (2009-2030)

In [10]:
fecha_inicio='2009-01-01'
fecha_fin='2030-12-31'
dataFrameDate = pd.DataFrame({'fecha': pd.date_range(fecha_inicio, fecha_fin)})
dataFrameDate['año'] = dataFrameDate.fecha.dt.year
dataFrameDate['mes'] = dataFrameDate.fecha.dt.month
dataFrameDate['dia'] = dataFrameDate.fecha.dt.day
dataFrameDate['nombre_dia'] = dataFrameDate.fecha.dt.day_name()
dataFrameDate['dia_semana'] = dataFrameDate.fecha.dt.dayofweek
dataFrameDate['semana'] = dataFrameDate.fecha.dt.weekofyear
dataFrameDate['trimestre'] = dataFrameDate.fecha.dt.quarter
dataFrameDate = dataFrameDate[['fecha', 'año', 'mes', 'dia', 'nombre_dia', 'dia_semana', 'semana', 'trimestre']] 
dataFrameDate

C:\Users\Joelv\AppData\Local\Temp/ipykernel_5692/259654942.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataFrameDate['semana'] = dataFrameDate.fecha.dt.weekofyear


,fecha,año,mes,dia,nombre_dia,dia_semana,semana,trimestre
0,2009-01-01,2009,1,1,Thursday,3,1,1
1,2009-01-02,2009,1,2,Friday,4,1,1
2,2009-01-03,2009,1,3,Saturday,5,1,1
3,2009-01-04,2009,1,4,Sunday,6,1,1
4,2009-01-05,2009,1,5,Monday,0,2,1
...,...,...,...,...,...,...,...,...
8030,2030-12-27,2030,12,27,Friday,4,52,4
8031,2030-12-28,2030,12,28,Saturday,5,52,4
8032,2030-12-29,2030,12,29,Sunday,6,52,4
8033,2030-12-30,2030,12,30,Monday,0,1,4


## Extracción de datos

In [11]:
Track=pd.read_sql_query("""SELECT  t.Name, mt.Name AS MediaType, g.Name AS Genre, t.Composer, t.Milliseconds, t.Bytes, t.UnitPrice 
       FROM tracks t
       INNER JOIN
       [genres] g ON t.GenreId = g.GenreId
       INNER JOIN
       media_types mt ON t.MediaTypeId = mt.MediaTypeId""" , con=engine.connect())

Artists=pd.read_sql_query("""SELECT Name From artists ; """ , con=engine.connect())
Customers=pd.read_sql_query("""SELECT  FirstName, LastName, Phone, Fax, Email from customers; """ , con=engine.connect())
InvoiceItem=pd.read_sql_query("""SELECT UnitPrice, Quantity from invoice_items ; """ , con=engine.connect())
Album=pd.read_sql_query("""SELECT Title From albums ; """ , con=engine.connect())
Location=pd.read_sql_query("""SELECT  BillingAddress AS Address, BillingCity AS City, BillingState AS State, BillingCountry AS Country, BillingPostalCode  AS PostalCode from invoices; """ , con=engine.connect())
Employee=pd.read_sql_query("""SELECT FirstName, LastName, Title, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email From employees ; """ , con=engine.connect())

TablaHechos = pd.read_sql_query("""SELECT invoice_items.InvoiceLineId as InvoicesID, customers.CustomerId AS CustomerID, employees.EmployeeId AS EmployeeID, dim_time.timeID AS TimeID,
invoices.InvoiceId as LocationID, tracks.TrackId AS TrackId, artists.ArtistId AS ArtistId, albums.AlbumId AS AlbumId, invoices.Total AS Total
FROM invoices INNER JOIN customers
ON invoices.CustomerId= customers.CustomerId
INNER JOIN employees
ON employees.EmployeeId= customers.SupportRepId
INNER JOIN invoice_items
ON invoice_items.InvoiceId= invoices.InvoiceId
INNER JOIN tracks
ON tracks.TrackId=  invoice_items.TrackId
INNER JOIN albums
ON tracks.AlbumId= albums.AlbumId
INNER JOIN artists 
ON albums.ArtistId= artists.ArtistId 
INNER JOIN dim_time
ON invoices.InvoiceDate= substr(dim_time.fecha,0,20);""", con=engine.connect())



## Transformación

In [12]:
frameTrack=pd.DataFrame(Track)
frameArtists=pd.DataFrame(Artists)
frameCustomers=pd.DataFrame(Customers)
frameInvoiceItem=pd.DataFrame(InvoiceItem)
frameAlbum=pd.DataFrame(Album)
frameEmployee=pd.DataFrame(Employee)
frameLocation=pd.DataFrame(Location)
frameTablaHechos=pd.DataFrame(TablaHechos)


frameTrack.fillna(method="ffill", inplace = True) 
frameArtists.fillna(method="ffill", inplace = True) 
frameCustomers.fillna(method="ffill", inplace = True) 
frameInvoiceItem.fillna(method="ffill", inplace = True) 
frameAlbum.fillna(method="ffill", inplace = True) 
frameEmployee.fillna(method="ffill", inplace = True) 
frameLocation.fillna(method="ffill", inplace = True) 
frameTablaHechos

,InvoicesID,CustomerID,EmployeeID,TimeID,LocationID,TrackId,ArtistId,AlbumId,Total
0,531,1,3,435,98,3247,158,253,3.98
1,532,1,3,435,98,3248,158,253,3.98
2,649,1,3,529,121,447,52,37,3.96
3,650,1,3,529,121,449,52,37,3.96
4,651,1,3,529,121,451,52,37,3.96
...,...,...,...,...,...,...,...,...,...
2235,1415,57,5,1150,262,1602,22,129,5.94
2236,1416,57,5,1150,262,1606,22,130,5.94
2237,1417,57,5,1150,262,1610,22,131,5.94
2238,1418,57,5,1150,262,1614,22,131,5.94


## Carga

### Carga del DataFrame de las fechas hacia la base de datos Chinook

In [6]:
dataFrameDate.to_sql(name='dim_time', con=engine.connect(), if_exists='append',index=False)

### Carga de los diferentes DataFrames hacia la base de datos DW_Sales_Music

In [13]:
conec = sqlalchemy.create_engine('mssql+pyodbc://JOELMOLINA/DW_Sales_Music?driver=SQL+Server+Native+Client+11.0')
try:
    frameTrack.to_sql(name='Dim_Track', con=conec, if_exists='append',index=False)
    frameArtists.to_sql(name='Dim_Artist', con=conec, if_exists='append',index=False)
    frameCustomers.to_sql(name='Dim_Customer', con=conec, if_exists='append',index=False)
    frameAlbum.to_sql(name='Dim_Album', con=conec, if_exists='append',index=False)
    frameEmployee.to_sql(name='Dim_Employee', con=conec, if_exists='append',index=False)
    frameLocation.to_sql(name='Dim_Location', con=conec, if_exists='append',index=False)
    dataFrameDate.to_sql(name='Dim_Time', con=conec, if_exists='append',index=False)
    frameInvoiceItem.to_sql(name='Dim_Invoice_Item', con=conec, if_exists='append',index=False)
    frameTablaHechos.to_sql(name='Fact_Sales', con=conec, if_exists='append',index=False)
    print("CARGA EXITOSA")
    
except Exception as ex:
    print("Ocurrió un error: {}".format(ex))

CARGA EXITOSA


In [14]:
Employee

,FirstName,LastName,Title,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,Andrew,Adams,General Manager,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,Nancy,Edwards,Sales Manager,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,Jane,Peacock,Sales Support Agent,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,Margaret,Park,Sales Support Agent,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,Steve,Johnson,Sales Support Agent,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,Michael,Mitchell,IT Manager,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,Robert,King,IT Staff,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
7,Laura,Callahan,IT Staff,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
